Name: Beyza Akdeniz

ID: 201101017

Course: YAP476 /BIL573

In [80]:
import pandas as pd
import os

In [81]:
path = "/home/beyza/Desktop/HW1/RetailSalesData"
product_hierarchy_path = path+'/product.csv'
store_cities_path = path+'/store.csv'
sales_csv_path = path+'/sales.csv'

### TASK-1: Load Data

In [82]:

product_df = pd.read_csv(product_hierarchy_path)
store_df = pd.read_csv(store_cities_path)
sales_df = pd.read_csv(sales_csv_path)


### TASK-2: Design and Implement Star Schema

Design the Star Schema
A typical star schema consists of a central fact table surrounded by dimension tables. 

Fact Table:
1. sales_fact: Stores sales data with foreign keys linking to dimension tables.
2. Measures: price, quantity_sold, stock.

Dimension Tables:
1. product_dim: Stores product details. 
2. store_dim: Stores store details.
3. time_dim: Stores date and time details.('date_id', 'date (text)', 'year', 'month', 'day') 

For time_dim:
1. Drop dublicate dates from sales_df
2. Convert the date column  to datetime; use pd.to_datetime() to convert the date column to a datetime object.
3. Extract date components for creating new columns for year, month, day, and keep the original date column as text and add index as date_id.
4. Insert data to time_dim table

In [83]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute('DROP TABLE IF EXISTS product_dim')
cursor.execute('DROP TABLE IF EXISTS store_dim')
cursor.execute('DROP TABLE IF EXISTS time_dim')
cursor.execute('DROP TABLE IF EXISTS sales_fact')

# Create dimension tables
cursor.execute('''
    CREATE TABLE product_dim (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        sub_category TEXT
    )
''')

cursor.execute('''
    CREATE TABLE store_dim (
        store_id INTEGER PRIMARY KEY,
        store_name TEXT,
        city_id TEXT,
        store_size INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE time_dim (
        date_id INTEGER PRIMARY KEY,
        date TEXT,
        year INTEGER,
        month INTEGER,
        day INTEGER
    )
''')

# Create fact table
cursor.execute('''
    CREATE TABLE sales_fact (
        sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        store_id INTEGER,
        date_id INTEGER,
        price DOUBLE,
        quantity_sold INTEGER,
        stock INTEGER,
        FOREIGN KEY (product_id) REFERENCES product_dim (product_id),
        FOREIGN KEY (store_id) REFERENCES store_dim (store_id),
        FOREIGN KEY (date_id) REFERENCES time_dim (date_id)
    )
''')

# Commit the changes
conn.commit()

sales_df['date'] = pd.to_datetime(sales_df['date'])

# Insert data into product_dim
product_df.to_sql('product_dim', conn, if_exists='replace', index=False)

# Insert data into store_dim
store_dim_df = store_df[['store_id', 'store_name', 'city_id', 'store_size']] 
store_dim_df.to_sql('store_dim', conn, if_exists='replace', index=False)

# Insert data into time_dim
time_df = sales_df[['date']].drop_duplicates().reset_index(drop=True)
time_df['year'] = time_df['date'].dt.year
time_df['month'] = time_df['date'].dt.month
time_df['day'] = time_df['date'].dt.day
time_df['date_id'] = time_df['date'].dt.strftime('%Y%m%d').astype('int64')

time_df['date'] = time_df['date'].dt.strftime('%Y-%m-%d %T')
time_dim_df = time_df[['date_id', 'date', 'year', 'month', 'day']]
time_dim_df.to_sql('time_dim', conn, if_exists='replace', index=False)

# Insert data into sales_fact
sales_df['date'] = sales_df['date'].dt.strftime('%Y-%m-%d %T')
# Check to avoid conflicts
if 'date_id' in sales_df.columns:
    sales_df.drop(columns=['date_id'], inplace=True)
# Merge sales_df with time_df to get date_id
sales_df = sales_df.merge(time_df[['date', 'date_id']], on='date', how='left')
sales_fact_df = sales_df[['product_id', 'store_id', 'date_id', 'price', 'sales', 'stock']]
# sales_fact_df['sales'] = sales_fact_df['sales'].astype(int)
# sales_fact_df['stock'] = sales_fact_df['stock'].astype(int)
sales_fact_df.insert(0, 'sale_id', range(0, len(sales_fact_df)))
# Rename columns
sales_fact_df = sales_fact_df.rename(columns={'sales': 'quantity_sold'})
sales_fact_df = sales_fact_df[['sale_id', 'product_id', 'store_id', 'date_id', 'price', 'quantity_sold', 'stock']].copy()
sales_fact_df.to_sql('sales_fact', conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()


Verify the Data

 Connect to database and display all  dimension tables and fact table, you can  show only first 5 input of tables

In [84]:
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()
    
# Function to display the first 5 rows of a table
def display_table(table_name):
    query = f"SELECT * FROM {table_name} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    display(df)

# Verify the data in each table
print("\nProduct Dimension:")
display_table('product_dim')

print("\nStore Dimension:")
display_table('store_dim')

print("\nTime Dimension:")
display_table('time_dim')

print("\nSales Fact:")
display_table('sales_fact')

# Close the connection
conn.close()


Product Dimension:


,product_id,product_name,category,sub_category
0,0,P0000,H00,H0004
1,1,P0001,H01,H0105
2,2,P0002,H03,H0315
3,3,P0004,H03,H0314
4,4,P0005,H03,H0312



Store Dimension:


,store_id,store_name,city_id,store_size
0,0,S0091,C013,19
1,1,S0012,C005,28
2,2,S0045,C008,17
3,3,S0032,C019,14
4,4,S0027,C022,24



Time Dimension:


,date_id,date,year,month,day
0,20170102,2017-01-02 00:00:00,2017,1,2
1,20170103,2017-01-03 00:00:00,2017,1,3
2,20170104,2017-01-04 00:00:00,2017,1,4
3,20170105,2017-01-05 00:00:00,2017,1,5
4,20170106,2017-01-06 00:00:00,2017,1,6



Sales Fact:


,sale_id,product_id,store_id,date_id,price,quantity_sold,stock
0,0,1,67,20170102,6.25,0.0,8.0
1,1,1,1,20170102,6.25,1.0,0.0
2,2,1,139,20170102,6.25,2.0,0.0
3,3,1,94,20170102,6.25,0.0,6.0
4,4,1,108,20170102,6.25,0.0,1.0


### TASK-3: Perform OLAP Operations

In [85]:
import sqlite3
import pandas as pd
path = "/home/beyza/Desktop/HW1/RetailSalesData"


# Reconnect to the database
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

Example Queries:

Roll-up: Aggregates data along a dimension. For example, aggregate stocks by month.

Drill-down: Opposite of roll-up. For example, break down sales(quantity_sold) by day.

Slice: Select a single dimension of data. For example, sales(quantity_sold) for a specific product (P0006).

Dice: Select multiple dimensions of data. For example, sales(quantity_sold) for specific products ('P0006','P00017') and stores ('S0091', 'S0027')

Pivot: Reorganize data along different dimensions. For example, pivot sales data by store and product.

Within the scope of this task, you will implement the OLAP operations given above as examples, in addition, you will define 1 example for each type of OLAP operation and you will implement them as well.

In [86]:

# Roll-up: Aggregate stocks by month
rollup_query = '''
SELECT 
    t.year,
    t.month,
    SUM(sf.stock) AS total_stock
FROM 
    sales_fact sf
JOIN 
    time_dim t ON sf.date_id = t.date_id
GROUP BY 
    t.year, t.month
ORDER BY 
    t.year, t.month;
'''
rollup_df = pd.read_sql_query(rollup_query, conn)
print("Roll-up (Aggregate sales by month):")
display(rollup_df)

# Drill-down: Break down sales by day
drilldown_query = '''
SELECT 
    t.year,
    t.month,
    t.day,
    SUM(sf.quantity_sold) AS total_quantity_sold
FROM 
    sales_fact sf
JOIN 
    time_dim t ON sf.date_id = t.date_id
GROUP BY 
    t.year, t.month, t.day
ORDER BY 
    t.year, t.month, t.day;
'''
drilldown_df = pd.read_sql_query(drilldown_query, conn)
print("\nDrill-down (Break down sales by day):")
display(drilldown_df)


# Slice: Sales for a specific product(P0006)
slice_query = '''
SELECT 
    t.year,
    t.month,
    p.product_name,
    SUM(sf.quantity_sold) AS total_sales
FROM 
    sales_fact sf
JOIN 
    time_dim t ON sf.date_id = t.date_id
JOIN 
    product_dim p ON sf.product_id = p.product_id
WHERE 
    p.product_name = 'P0006'
ORDER BY 
    t.date;
'''
slice_df = pd.read_sql_query(slice_query, conn)
print("\nSlice (Sales for a specific product P0006):")
display(slice_df)


# Dice: sales(quantity_sold) for specific products ('P0006','P0017') and stores ('S0091', 'S0027')
dice_query = '''
SELECT 
    t.year,
    t.month,
    p.product_name,
    s.store_name,
    SUM(sf.quantity_sold) AS total_sales
FROM 
    sales_fact sf
JOIN 
    time_dim t ON sf.date_id = t.date_id
JOIN 
    product_dim p ON sf.product_id = p.product_id
JOIN 
    store_dim s ON sf.store_id = s.store_id
WHERE 
    p.product_name IN ('P0006', 'P0017')
    AND s.store_name IN ('S0091', 'S0027')
GROUP BY 
    p.product_name, s.store_name
ORDER BY 
    t.date;
'''
dice_df = pd.read_sql_query(dice_query, conn)
print("\nDice (Sales for specific products and stores):")
display(dice_df)



# Pivot: Sales by store and product
pivot_query = '''
SELECT 
    sf.store_id,
    sf.product_id,
    SUM(sf.quantity_sold) AS total_quantity_sold
FROM 
    sales_fact sf
GROUP BY 
    sf.store_id, sf.product_id
ORDER BY 
    sf.store_id, sf.product_id;
'''
pivot_df = pd.read_sql_query(pivot_query, conn)
print("\nPivot (Sales by store and product):")
display(pivot_df)

# Close the connection
conn.close()


Roll-up (Aggregate sales by month):


,year,month,total_stock
0,2017,1,6.458541e+06
1,2017,2,6.096165e+06
2,2017,3,6.809325e+06
3,2017,4,6.621139e+06
4,2017,5,7.040010e+06
5,2017,6,7.132609e+06
6,2017,7,7.327113e+06
7,2017,8,8.170714e+06
8,2017,9,8.008085e+06
9,2017,10,8.350998e+06



Drill-down (Break down sales by day):


,year,month,day,total_quantity_sold
0,2017,1,2,6817.962
1,2017,1,3,6578.220
2,2017,1,4,6615.686
3,2017,1,5,6440.778
4,2017,1,6,7182.927
...,...,...,...,...
1087,2019,12,25,NaN
1088,2019,12,26,NaN
1089,2019,12,27,NaN
1090,2019,12,28,NaN



Slice (Sales for a specific product P0006):


,year,month,product_name,total_sales
0,2017,1,P0006,2872.611



Dice (Sales for specific products and stores):


,year,month,product_name,store_name,total_sales
0,2017,1,P0006,S0027,9.0
1,2017,1,P0006,S0091,14.0
2,2017,1,P0017,S0027,289.0
3,2017,1,P0017,S0091,236.0



Pivot (Sales by store and product):


,store_id,product_id,total_quantity_sold
0,0,1,48.0
1,0,2,2.0
2,0,3,9.0
3,0,4,94.0
4,0,5,14.0
...,...,...,...
85220,143,638,0.0
85221,143,639,0.0
85222,143,640,0.0
85223,143,641,0.0


### Task 4: Join Indexing and Performance Comparison

Measure the performance of the OLAP queries with and without indexes using Python's time module.

STEPS:

1. Create join indexes.

2. Measure the performance of the OLAP queries with the indexes.

3. Drop the indexes.

4. Measure the performance of the OLAP queries without the indexes.

4. Compare the performance and calculate the improvement.

Create an index that combines these columns to speed up the join operations.

Example: In the context of our data warehouse, we have a sales_fact table that joins with product_dim, store_dim, and time_dim. We can create join indexes on these columns.

These cresated indexes will help optimize queries that involve joining the sales_fact table with the product_dim, store_dim, and time_dim tables on the specified columns.


In [87]:
import sqlite3
import time

def execute_query(query, conn):
    start_time = time.time()
    df = pd.read_sql_query(query, conn)
    end_time = time.time()
    return df, end_time - start_time

# Reconnect to the database
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

# Drop indexes
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_date_id;')
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_product_id;')
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_store_id;')
conn.commit()

# Create join indexes
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_fact_date_id ON sales_fact (date_id);')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_fact_product_id ON sales_fact (product_id);')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_fact_store_id ON sales_fact (store_id);')
conn.commit()

# Define queries
queries = {
    'rollup': '''
        SELECT t.year, t.month, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact s
        JOIN time_dim t ON s.date_id = t.date_id
        GROUP BY t.year, t.month
        ORDER BY t.year, t.month;
    ''',
    'drilldown': '''
        SELECT t.year, t.month, t.day, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact s
        JOIN time_dim t ON s.date_id = t.date_id
        GROUP BY t.year, t.month, t.day
        ORDER BY t.year, t.month, t.day;
    ''',
    'slice': '''
        SELECT t.year, t.month, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact s
        JOIN time_dim t ON s.date_id = t.date_id
        JOIN product_dim p ON s.product_id = p.product_id
        WHERE p.product_name = 'Specific Product'
        GROUP BY t.year, t.month
        ORDER BY t.year, t.month;
    ''',
    'dice': '''
        SELECT t.year, t.month, p.product_name, st.store_name, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact s
        JOIN time_dim t ON s.date_id = t.date_id
        JOIN product_dim p ON s.product_id = p.product_id
        JOIN store_dim st ON s.store_id = st.store_id
        WHERE p.product_name IN ('Product A', 'Product B') AND st.store_name IN ('Store X', 'Store Y')
        GROUP BY t.year, t.month, p.product_name, st.store_name
        ORDER BY t.year, t.month, p.product_name, st.store_name;
    ''',
    'pivot': '''
        SELECT st.store_name, p.product_name, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact s
        JOIN store_dim st ON s.store_id = st.store_id
        JOIN product_dim p ON s.product_id = p.product_id
        GROUP BY st.store_name, p.product_name
        ORDER BY st.store_name, p.product_name;
    '''
}

# Measure performance with indexes
performance_with_indexes ={}
for query_name, query in queries.items():
    _, execution_time = execute_query(query, conn)
    performance_with_indexes[query_name] = execution_time

# Drop indexes
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_date_id;')
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_product_id;')
cursor.execute('DROP INDEX IF EXISTS idx_sales_fact_store_id;')
conn.commit()

# Measure performance without indexes
performance_without_indexes = {}
for query_name, query in queries.items():
    _, execution_time = execute_query(query, conn)
    performance_without_indexes[query_name] = execution_time

# Close the connection
conn.close()

# Compare performances
performance_comparison = pd.DataFrame({
    'With Indexes': performance_with_indexes,
    'Without Indexes': performance_without_indexes
})

performance_comparison['Improvement'] = performance_comparison['Without Indexes'] / performance_comparison['With Indexes']
print("Performance Comparison:")
print(performance_comparison)


Performance Comparison:
           With Indexes  Without Indexes  Improvement
rollup         7.379844         8.971834     1.215721
drilldown      8.641819        10.283846     1.190009
slice          0.000695         0.000624     0.897427
dice           0.000557         0.000524     0.940899
pivot         26.868277        14.440809     0.537467
